In [1]:
import pandas as pd
import utils as keu # key_electra_utils.py 참고
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('monologg/koelectra-base-v3-discriminator')

raw_data = pd.read_csv('data/raw_book_info_list.csv',index_col=0)



No sentence-transformers model found with name /Users/yangwoolee/.cache/torch/sentence_transformers/monologg_koelectra-base-v3-discriminator. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/yangwoolee/.cache/torch/sentence_transformers/monologg_koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mo

### SBert를 활용해 Sentence embedding을 만들때 모델의 Max_seq_len에 영향을 받을까?

**궁금한 점**

* koelectra 경우 Max_seq_len가 512이므로, 토큰이 512개 이상인 문장은 512개 이후에 존재하는 토큰은 버림

* 따라서 문장 하나의 토큰이 512개가 넘으면 앞의 512개의 토큰으로만 sentence embedding을 수행할 것으로 생각

* 지금 사용하는 input 데이터는 여러 문장을 하나의 문장으로 합친 다음 사용하므로 모든 문장의 토큰이 512개가 넘음

* 이런 이유로 512개 이상의 토큰을 가진 문장의 순서를 바꿈에 따라 출력 결과도 바뀔 것으로 예상.

**증명방법**

* 지금 사용하는 데이터의 [book_title, book_toc, book_intro, publisher] column은 4종류이며, 4종류의 문장을 하나로 통합해서 하나의 input data로 사용

* '파이토치 딥러닝 프로젝트 모음집' 도서의 경우 총 793개의 token으로 구분됨. 개별 column의 token 개수는 각 [5, 316, 277,195] 임.

* max_length 512이므로 column 순서로 문장을 합치면 각각 [5, 316, 191,0]개의 토큰을 활용할 것이라 생각

* 이를 대조하기 위해 원래의 column 순서를 변경해 [5, 316, 277, 195]의 토큰 개수를 [5, 277, 316, 195]로 변경하여 문장 생성, 512개까지 포함하면 [5, 277, 230, 0]개의 토큰으로 학습할 것이라 생각

* 기존 문장을 original_sentence, 새로운 문장은 new_sentence로 지칭하였음

* origin_sentence 출력과 new_sentence 출력이 다를 경우 SBert의 sentence_embedding이 모델 max_seq_len에 영향을 받는 다 생각할 수 있음.

* origin_sentence 출력과 new_sentence 출력이 같을 경우 어떻게 sentence embedding을 만드는지 추가적인 학습이 필요

**결론**
* origin_sentence 출력과 new_sentence 출력이 같음
* Sentence embedding은 모델 max_seq_len에 영향받지 않음

### Original Sentence 만들기

In [11]:
original_data = raw_data.iloc[1100] # 1100 파이토치 딥러닝 프로젝트 모음집

book_info :str =keu.Merge_Series_to_str(original_data)
book_info_trans = keu.trans_engwords_to_hanwords(book_info)

original_sentence = keu.find_han_words(' '.join(book_info_trans))

변환한 도서정보 :  파이토치 딥러닝 프로젝트 모음집


### New sentence 만들기

In [6]:
new_data = raw_data.iloc[1100].values.tolist()
new_data.append(new_data.pop(1))

book_info :str =keu.Merge_Series_to_str(new_data)
book_info_trans = keu.trans_engwords_to_hanwords(book_info)

new_sentence = keu.find_han_words(' '.join(book_info_trans))

변환한 도서정보 :  파이토치 딥러닝 프로젝트 모음집


### Original sentence vs New sentence 비교

In [13]:
print('----original----')
print(original_sentence[10:50])

print('----new----')
print(new_sentence[10:50])

----original----
['인공지능', '사례', '머신러닝', '머신', '러닝', '머신러닝이란', '머신러닝', '구분', '지도학습', '러닝', '비지도학습', '러닝', '과적합과', '모델', '학습법', '성능', '지표', '딥러닝', '딥', '러닝', '딥러닝이란', '딥러닝', '발전', '과정', '퍼셉트론', '퍼셉트론', '다층', '퍼셉트론', '레이어', '퍼셉트론', '인공신경망', '핵심', '알고리즘', '고급', '딥러닝', '기술', '뉴럴', '네트워크', '뉴럴', '네트워크']
----new----
['실제로', '우리가', '흔히', '다루는', '날것의', '데이터를', '활용한', '프로젝트', '예제집은', '찾기가', '어렵습니다', '이', '책은', '딥러닝', '프로젝트들을', '중점적으로', '다룸으로써', '어느정도', '딥러닝', '지식은', '있으나', '프로젝트로', '경험하고', '싶은', '독자들에게', '꼭', '필요한', '책이', '될', '것입니다', '이', '책은', '크게', '이론', '파트와', '실전', '파트로', '나누어져', '있습니다', '이론']


### 결과 비교
문장 순서를 바꾸더라도 결과는 같음

In [18]:
origin = keu.key_extraction(original_sentence,model)
new = keu.key_extraction(new_sentence,model)
print('----original----')
print(sorted(origin.index.tolist()))
print('----new----')
print(sorted(new.index.tolist()))

----original----
['국민청원', '다양한', '데이터', '데이터를', '딥러닝', '딥러닝을', '러닝', '분류하기', '불러오기', '이론', '있습니다', '전처리', '책으로', '책은', '크롤링', '파이토치', '파헤치기', '퍼셉트론', '프로젝트', '프로젝트로']
----new----
['국민청원', '다양한', '데이터', '데이터를', '딥러닝', '딥러닝을', '러닝', '분류하기', '불러오기', '이론', '있습니다', '전처리', '책으로', '책은', '크롤링', '파이토치', '파헤치기', '퍼셉트론', '프로젝트로', '프로젝트를']


## Electra

<br/>

### Pre-trained 된 generator를 가지고 어떻게 fake sentence를 만들 수 있을까?

**궁금한점**
* koElectra로 Domain Adaptation을 위한 설계중 어떻게 generator로 fake sentence를 만들어야 하는징 대한 문제에 직면했음

* 무엇보다 Generator는 Bert와 동일한 구조라 생각해 output이 (1,src_token_len, vocab_size)이 나와야 된다고 생각했는데, output shape이 (1,src_token_len, 256)으로 나오는게 의아했음. 

* huggingface 페이지에 작성한 활용 예시대로 활용할 경우 문제없이 작동하길래 어디서부터 잘못된 것인지 파악이 필요했음.

**증명방법**

* (1,src_token_len, 256)를 (1,src_token_len, vocab_size)로 변환하기 위해 새로운 모델을 제작

* transformers pipeline module을 뜯어보고 어떻게 output을 생성하는지 추적

**배운점**

* Encoder output을 vocab_size 크기로 변경하는 마지막 layer 또한 학습이 필요함을 경험으로 확인하였음.
    * last_hidden_state를 vocab_size 차원으로 확대 한 뒤 사용해보니 그 결과가 엉망이었음.

* Transformers 라이브러리에서 불러오는 클래스에 따라 모델의 Output이 달라짐을 이해했음.

    * Electramodel을 사용할 경우 Encoder 끝단의 output을 출력함. 출력명은 `last_hidden_state`, 크기는 `(1,src_token_len, 256)`임.
    * ElectraForMaskedLM을 사용할 경우 vocab 단어별 확률을 출력함. 출력명은 `logits` 크기는 `(1,src_token_len, vocab_size)` 임.


### 예제 불러오기

In [1]:
from transformers import pipeline, ElectraTokenizer,ElectraModel
import torch
import torch.nn as nn
from pprint import pprint
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

raw_str = '조건검색 실시간으로 [MASK]하기'
input_data = tokenizer(raw_str,return_tensors='pt')

pprint({'tokenizing' : tokenizer.tokenize(raw_str)})
pprint({'model_input' : input_data['input_ids']})



{'tokenizing': ['조건', '##검', '##색', '실시간', '##으로', '[MASK]', '하', '##기']}
{'model_input': tensor([[    2,  7080,  4166,  4703, 10460, 10749,     4,  3755,  4031,     3]])}


### ElectraModel을 활용하여 [Mask]를 예측할 경우

### 부자연스러운 결과 생성

In [16]:
generator = ElectraModel.from_pretrained("monologg/koelectra-base-v3-generator")

token_output = generator(**input_data)

print('Model_output')
print('last_hidden_state : ',token_output.last_hidden_state.shape) 


class output_generator(nn.Module) :
    '''
    last_hidden_state를 vocab_size에 맞게 차원 확장하는 모델 구현
    input_size = (1,src_token_size, 256) 
    output_size = (1,src_token_size, 35000)
    '''
    def __init__(self, output_embed : int, vocab_size : int) -> None:
        super().__init__()
        self.output_embed = output_embed
        self.vocab_embedding = nn.Linear(output_embed,vocab_size) #(256 => 35000)

    def forward(self,src) :
        return self.vocab_embedding(src) #(src_token_len, 35000)


token_logits = output_generator(256,35000) # (embed_size, vocab_size)
token_logits = token_logits(token_output.last_hidden_state)

# [mask]가 위치한 idx
mask_token_index = torch.where(input_data["input_ids"] == tokenizer.mask_token_id)[1]

# [mask]가 위치한 vocab
mask_token_logits = token_logits[0,mask_token_index-1, :]

# top 5추출
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

# 결과 추출
print('')
print("Input_data ", raw_str)
print('-'*20)
for token in top_5_tokens:
    print(f"'>>> {raw_str.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

Model_output
last_hidden_state :  torch.Size([1, 10, 256])

Input_data  조건검색 실시간으로 [MASK]하기
--------------------
'>>> 조건검색 실시간으로 ##ition하기'
'>>> 조건검색 실시간으로 가느다란하기'
'>>> 조건검색 실시간으로 〈하기'
'>>> 조건검색 실시간으로 ##잡이하기'
'>>> 조건검색 실시간으로 연예하기'


### Hugging face generator 페이지에 나온 예제 활용 하는 경우(pipeline 활용)

### 정상 작동

In [8]:
fill_mask = pipeline(
    "fill-mask",
    model="monologg/koelectra-base-v3-generator",
    tokenizer="monologg/koelectra-base-v3-generator"
)

pprint(fill_mask(raw_str))

[{'score': 0.09738744050264359,
  'sequence': '조건검색 실시간으로 확인 하기',
  'token': 6465,
  'token_str': '확인'},
 {'score': 0.07290185242891312,
  'sequence': '조건검색 실시간으로 답변 하기',
  'token': 8610,
  'token_str': '답변'},
 {'score': 0.03160829842090607,
  'sequence': '조건검색 실시간으로 진행 하기',
  'token': 6355,
  'token_str': '진행'},
 {'score': 0.0188735481351614,
  'sequence': '조건검색 실시간으로 로그인 하기',
  'token': 22864,
  'token_str': '로그인'},
 {'score': 0.017551911994814873,
  'sequence': '조건검색 실시간으로 답 하기',
  'token': 2358,
  'token_str': '답'}]


### Transformers class를 ElectraModel에서 ElectraForMaskedLM로 바꿔 사용한 경우

### 예제와 같이 정상 작동

In [18]:
from transformers import ElectraForMaskedLM

forMaskedLM = ElectraForMaskedLM.from_pretrained('monologg/koelectra-base-v3-generator')

In [23]:
raw_str = '조건검색 실시간으로 [MASK]하기'
input_data = tokenizer(raw_str,return_tensors='pt')


logits = forMaskedLM(**input_data).logits

print('Model_output')
print('logits : ',logits.shape) 


mask_token_index = torch.where(input_data["input_ids"] == tokenizer.mask_token_id)[1]



mask_token_logits = logits[0,mask_token_index, :]


top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

print('')
print("Input_data ", raw_str)
print('-'*20)

for token in top_5_tokens:
    print(f"'>>> {raw_str.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

Model_output
logits :  torch.Size([1, 10, 35000])

Input_data  조건검색 실시간으로 [MASK]하기
--------------------
'>>> 조건검색 실시간으로 확인하기'
'>>> 조건검색 실시간으로 답변하기'
'>>> 조건검색 실시간으로 진행하기'
'>>> 조건검색 실시간으로 로그인하기'
'>>> 조건검색 실시간으로 답하기'


### Cuda Out of Memory 문제 해결 : 데이터 전처리를 제대로 해야하는 이유

* GPU 메모리가 급증하는 시기 발견하여 원인을 분석함.

* input_data의 shape를 추적한 결과 max_length(=512)에 근접할 경우 OOM이 발생함을 확인함.
    ```python
    200번째 : torch.Size([16, 450])
    ---------------------------------------------------------------------------
    OutOfMemoryError                          Traceback (most recent call last)
    ```

* 지금껏 batch_size에 OOM 문제 발생 시 batch_size를 줄이는데만 신경썼음.

* tokenizing 단계에서 max_length를 512->256으로 줄이면 쉽게 문제 해결이 가능했으나,

* max_length를 최대한 줄여 batch_size를 향상시킬 수 있는 방법을 고민함.


In [10]:
import pandas as pd
from transformers import ElectraTokenizer

# 전처리 된(줄 알았던) 데이터 로드
raw_data = pd.read_csv('data/old_data.csv',index_col=0)

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

raw_data.head(1)

,0
0,실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선 이...


### 최대 Token 개수 확인
* 최대 2081개의 토큰이 생성 가능함을 확인

In [12]:
tokens = tokenizer(raw_data['0'].tolist())
tokens_num = [len(i) for i in tokens['input_ids']]
df_token_num = pd.DataFrame(tokens_num)

df_token_num = df_token_num.sort_values(by=0,ascending=False)

df_token_num.head(100)

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


,0
165865,4456
108710,4353
170827,2487
116711,2478
116735,2081
...,...
138205,555
136695,550
163103,549
155702,533


### 토큰이 가장 많이 생긴 문장을 찾아본 결과 전처리 수행이 미흡함을 발견

**4456개 토큰으로 나눠지는 문장 원본**

> "id"" : ""SE-ebbe7d35-8e9b-4578-992c-989ae9a3a691"", ""src"" : ""http://mblogthumb4.phinf.naver.net/MjAyMDA5MjZfMjkg/MDAxNjAxMDgxNDc0NDc3.nPQjxQCHTJAkIPQrIGovUhhWVsuwnSUCGseyJu4exzEg.pJRoLx5cFKZ8t-lyDDg9q86j_bC9FTfagyWz8sPI-ecg.JPEG.0517park/SE-ebbe7d35-8e9b-4578-992c-989ae9a3a691.jpg"", ""linkUse"" : ""false"", ""link"" : """"}"" style=""color: #000000; text-decoration: none; cursor: pointer; -webkit-tap-highlight-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border: 0px; font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline;"" target=""_blank""&gt; id"" : ""SE-24ea1b1a-cb11-4f7e-8359-8786b04b1a32"", ""src"" : ""http://mblogthumb2.phinf.naver.net/MjAyMDA5MjZfMTAx/MDAxNjAxMDgxNDc1MjUx.JWFkCOG6se16qMJ9Cf6pbQ-If0wAk7LBtAnuiDVh3b4g.0RKV2pdWZCSg2weiS96XatrKZriqHfNPztptag4na8og.JPEG.0517park/SE-24ea1b1a-cb11-4f7e-8359-8786b04b1a32.jpg"", ""linkUse"" : ""false"", ""link"" : """"}"" style=""color: #000000; text-decoration: none; cursor: pointer; -webkit-tap-highlight-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border: 0px; font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline;"" target=""_blank""&gt; id"" : ""SE-e66e541b-4f5d-4bba-9232-6dc1bebd0c07"", ""src"" : ""http://mblogthumb2.phinf.naver.net/MjAyMDA5MjZfMjI3/MDAxNjAxMDgxNDc1Nzk0.XaPVoOK9D5XdvkFtH5VkaUC8Cw6PpI2qDWo-OQeBb5Mg.eJ8l6E-B4RU_alQ_-uw5fkQbk5km7ZXyTSvsITnYazQg.JPEG.0517park/SE-e66e541b-4f5d-4bba-9232-6dc1bebd0c07.jpg"", ""linkUse"" : ""false"", ""link"" : """"}"" style=""color: #000000; text-decoration: none; cursor: pointer; -webkit-tap-highlight-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border: 0px; font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline;"" target=""_blank""&gt; id"" : ""SE-7f93752e-f031-46d7-9c02-978d64ff9eba"", ""src"" : ""http://mblogthumb2.phinf.naver.net/MjAyMDA5MjZfMjM2/MDAxNjAxMDgxNDc2NDkw.LOVPbB03U1tCG2ruFWuHrq_nEVMB3coj5LMcQ7uTMz0g.GH4Mn-N15VzputfCixZy9eE1NQ76GoZU6Y_BVhj5viEg.JPEG.0517park/SE-7f93752e-f031-46d7-9c02-978d64ff9eba.jpg"", ""linkUse"" : ""false"", ""link"" : """"}"" style=""color: #000000; text-decoration: none; cursor: pointer; -webkit-tap-highlight-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border: 0px; font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline;"" target=""_blank""&gt; id"" : ""SE-d2a77837-52d9-4e92-b1b0-c336cf94747f"", ""src"" : ""http://mblogthumb1.phinf.naver.net/MjAyMDA5MjZfMjQ3/MDAxNjAxMDgxNDc3MDc5.nd5_sbsTXeuQY-V_tR6b03Ufdy3Bb4ccgOEzmuT8pd8g.Gn28Ki7kKe2gwvCSTvmj3qWX7MEBRuPAw8R6nwEH6Pcg.JPEG.0517park/SE-d2a77837-52d9-4e92-b1b0-c336cf94747f.jpg"", ""linkUse"" : ""false"", ""link"" : """"}"" style=""color: #000000; text-decoration: none; cursor: pointer; -webkit-tap-highlight-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border: 0px; font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline;"" target=""_blank""&gt; 다양한 브러시로 그림그리기 연습!"



### 데이터 전처리 재진행

In [164]:
# 전처리 수행하지 않은 raw_file 불러오기

book_reply = pd.read_csv('data/raw_book_reply.csv',index_col= 0)

book_reply['2'].head(2)

0    [['점프 투 파이썬 책의 커리큘럼처럼 이어져서 좋다.', '다만, 사실상 거의 입...
2    [['쉽게 설명되어 있어 입문용으로 좋습니다.'], ['도커를 배울 수 있는 최신 ...
Name: 2, dtype: object

### 리스트 내 리스트 제거를 위한 1단계 전처리

In [165]:
import ast 

sentences = book_reply[book_reply['2'].isna() == False]['2'].tolist()

# str type의 list를 다시 list type으로 전환 
# ** list를 csv로 저정할 경우 str type으로 저장되기 때문에 아래의 매서드 수행
sentences = list(map(lambda x : ast.literal_eval(x), sentences))

# 리스트 내 리스트 제거하기
book_reply = [j for i in sentences for j in i]
book_reply = [j for i in book_reply for j in i]

book_reply[0]

'점프 투 파이썬 책의 커리큘럼처럼 이어져서 좋다.'

### 문장 최대길이 확인

In [167]:
check_len = pd.DataFrame([len(j) for j in book_reply]).sort_values(by=0,ascending=False)
check_len

,0
9399,8167
66554,6854
71516,4896
17400,3926
71522,3697
...,...
93815,1
29683,1
29700,1
98607,1


### 최대 길이를 가진 문장 확인

In [170]:
book_reply = pd.DataFrame(book_reply)

max_sen = book_reply.iloc[9399].item()

max_sen


'lt;div class="se-component se-sticker se-l-default" id="SE-90d7bec4-b38e-4aee-a156-9b16e71b1447" style="margin: 20px 0px 0px; padding: 0px; border: 0px; font-variant-numeric: inherit; font-variant-east-asian: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline; position: relative;"&gt; &lt;div class="se-component-content" style="margin: 0px auto; padding: 0px 40px; border: 0px; font-variant-numeric: inherit; font-variant-east-asian: inherit; font-stretch: inherit; line-height: inherit; vertical-align: baseline; max-width: 100%;"&gt; &lt;/div&gt; &lt;/div&gt; 사실 저같은 경우에는 수학적인 부분이 상당히 약하고 자신없어서 피할 수 있으면 최대한 피하고 살고 싶었는데요. 아이를 키우고 공부를 시키면서 수학이 우리 생활과 얼마나 밀접하게 연관이 되어 있는지를 알게 되었고요. 아이 문제풀이를 위해 엄마도 수학문제를 풀기 시작했고 무조건 거부하고 피하면 안되겠다는 생각이 들더라고요. 엑셀역시 마찬가지로 기본적인 것들은 할 수 있지만 아주 소소한 부분에 그치고 있고.. 진짜 제가 하고 싶고 배우고 싶은 것은 엑셀함수를 자유자재로 활용하여 유용하게 시간활용을 하고 효율적으로 일처리를 하는 것이서어ㅛ. 한정희 쌤의 책을 만나보게 되었는데 잘 선택했다는 생각이 들더라고요. 저도 구독완료! &lt;div class="se-component se-sticker se-l-default" id="

### Regex로 전처리 테스트

* +는 1개 이상 매치, * 0개 이상 매치되는 경우를 의미함. 이를 활용해 원하는 값을 추출할 수 있음.

<br/>    


```python 

re.findall('\s*\w*\s*[\u3130-\u318F\uAC00-\uD7A3]+\s*\w*[.]*',max_sen)

```

* 위 regex를 풀어 설명하면 

* \s* : 0개 이상의 띄어쓰기를 가지고 있으며(띄어쓰기부터 시작해도 된다는 의미)
* \w* : 0개 이상의 영어 또는 숫자이며(영어부터 시작해도 된다는 의미)
* [\u3130-\u318F\uAC00-\uD7A3]+ : 1개 이상의 한글이며(최소 1개 이상의 한글은 들어가야한다는 의미)
* \s* : 0개 이상의 띄어쓰기를 가지고 있으며 
* \w* : 0개 이상의 영어 또는 숫자이며
* [.]* : 0개 이상의 마침표를 가진 경우를 
* 모두 찾는다.


In [174]:
import re 

re_result = re.findall('\s*\w*\s*[\u3130-\u318F\uAC00-\uD7A3]+\s*\w*[.]*',max_sen)

print(''.join(re_result))

 사실 저같은 경우에는 수학적인 부분이 상당히 약하고 자신없어서 피할 수 있으면 최대한 피하고 살고 싶었는데요. 아이를 키우고 공부를 시키면서 수학이 우리 생활과 얼마나 밀접하게 연관이 되어 있는지를 알게 되었고요. 아이 문제풀이를 위해 엄마도 수학문제를 풀기 시작했고 무조건 거부하고 피하면 안되겠다는 생각이 들더라고요. 엑셀역시 마찬가지로 기본적인 것들은 할 수 있지만 아주 소소한 부분에 그치고 있고.. 진짜 제가 하고 싶고 배우고 싶은 것은 엑셀함수를 자유자재로 활용하여 유용하게 시간활용을 하고 효율적으로 일처리를 하는 것이서어ㅛ. 한정희 쌤의 책을 만나보게 되었는데 잘 선택했다는 생각이 들더라고요. 저도 구독완료 특히 이 책의 저자님은 유튜브에 짤막한 강좌를 업로드하신 터라 무료 동영상 160강도 함께 만나볼 수 있어 좋은데요. 책을 보니 책만으로도 충분히 훌륭하고 좋다는 생각이 들어서 엑셀을 공부할 때에 다른 곳에 한눈팔지 말고 이 책과 저자님의 동영상강의를 보면 되겠다 싶더라고요. 책의 앞쪽에는 서두르지 마라 그러나 멈추지도 마라.라는 요한 볼프강 폰 괴테의 명언이 적혀 있어서요. 다둥맘인지라 서두를 수 없는 저에게 뭔가 의미심장하게 다가왔는데.. 한 권 한 권 이지스퍼블리싱에서 만나보는 된다 시리즈들로 제 인생이 조금씩 긍정적인 방향으로 바뀌고 있지 않나 싶습니다. 이 책은 총 셋째 마당으로 이뤄져 있으며.. 첫째 마당 엑셀의 기본 데이터 입력과 관리 둘째 마당 엑셀의 꽃 수식과 함수 셋째 마당 보고서에 필수 데이터 집계와 시각화 의 순서로 엑셀정복을 하게 되는데요. 이 책에서는 바쁜 직장인을 위한 7일 속성 입문 코스는 물론.. 16일 코스 스케줄도 수록하고 있어서 원하시는대로 진도를 나가보시면 좋을 듯  7일 속성 입문 코스로는 1일 차  업무에 많이 쓰인느 양식 만들며 기본 익히기 2일 차 데이터베이스 관리 및 유효성 검사 3일 차엑셀의 꽃 수식과 함수의 기본 4일 차 IF함수와 VLOOKUP함수 5일 차 데이터 요약과 필터링 6일 차  차트 

### pandas를 이용해 문장 전체를 전처리

In [182]:
regex_result = book_reply[0].str.findall('\s*\w*\s*[\u3130-\u318F\uAC00-\uD7A3]+\s*\w*[.]*')
print(regex_result[:2])

regex_result = regex_result.apply(lambda x : ''.join(x))
print(regex_result[:2])


0                    [점프 투 파이썬,  책의 커리큘럼처럼 이어져서,  좋다.]
1    [다만,  사실상 거의 입문,  표준인 파이참이나 vscode,  로 에디터를 사용...
Name: 0, dtype: object
0                          점프 투 파이썬 책의 커리큘럼처럼 이어져서 좋다.
1    다만 사실상 거의 입문 표준인 파이참이나 vscode 로 에디터를 사용했으면 좋았겠...
Name: 0, dtype: object


### 문장별 길이 재확인 

In [190]:
regex_result_len = regex_result.str.len().sort_values(ascending=False)

regex_result_len

9399     2190
92987    1699
99430    1515
48162    1090
66554    1029
         ... 
93477       0
15525       0
63215       0
15498       0
93022       0
Name: 0, Length: 102367, dtype: int64

### 최대 길이를 가진 문장 확인 

In [191]:
regex_result.iloc[9399]

' 사실 저같은 경우에는 수학적인 부분이 상당히 약하고 자신없어서 피할 수 있으면 최대한 피하고 살고 싶었는데요. 아이를 키우고 공부를 시키면서 수학이 우리 생활과 얼마나 밀접하게 연관이 되어 있는지를 알게 되었고요. 아이 문제풀이를 위해 엄마도 수학문제를 풀기 시작했고 무조건 거부하고 피하면 안되겠다는 생각이 들더라고요. 엑셀역시 마찬가지로 기본적인 것들은 할 수 있지만 아주 소소한 부분에 그치고 있고.. 진짜 제가 하고 싶고 배우고 싶은 것은 엑셀함수를 자유자재로 활용하여 유용하게 시간활용을 하고 효율적으로 일처리를 하는 것이서어ㅛ. 한정희 쌤의 책을 만나보게 되었는데 잘 선택했다는 생각이 들더라고요. 저도 구독완료 특히 이 책의 저자님은 유튜브에 짤막한 강좌를 업로드하신 터라 무료 동영상 160강도 함께 만나볼 수 있어 좋은데요. 책을 보니 책만으로도 충분히 훌륭하고 좋다는 생각이 들어서 엑셀을 공부할 때에 다른 곳에 한눈팔지 말고 이 책과 저자님의 동영상강의를 보면 되겠다 싶더라고요. 책의 앞쪽에는 서두르지 마라 그러나 멈추지도 마라.라는 요한 볼프강 폰 괴테의 명언이 적혀 있어서요. 다둥맘인지라 서두를 수 없는 저에게 뭔가 의미심장하게 다가왔는데.. 한 권 한 권 이지스퍼블리싱에서 만나보는 된다 시리즈들로 제 인생이 조금씩 긍정적인 방향으로 바뀌고 있지 않나 싶습니다. 이 책은 총 셋째 마당으로 이뤄져 있으며.. 첫째 마당 엑셀의 기본 데이터 입력과 관리 둘째 마당 엑셀의 꽃 수식과 함수 셋째 마당 보고서에 필수 데이터 집계와 시각화 의 순서로 엑셀정복을 하게 되는데요. 이 책에서는 바쁜 직장인을 위한 7일 속성 입문 코스는 물론.. 16일 코스 스케줄도 수록하고 있어서 원하시는대로 진도를 나가보시면 좋을 듯  7일 속성 입문 코스로는 1일 차  업무에 많이 쓰인느 양식 만들며 기본 익히기 2일 차 데이터베이스 관리 및 유효성 검사 3일 차엑셀의 꽃 수식과 함수의 기본 4일 차 IF함수와 VLOOKUP함수 5일 차 데이터 요약과 필터링 6일 차  차트

### 최대 Token 개수를 줄이기 위해 문장 나누기

* kiwi 라이브러리 활용


In [192]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

sep_result = regex_result.apply(lambda x : kiwi.split_into_sents(x))

new_sep_result = []
for i in sep_result.tolist() :
    for j in i :
        new_sep_result.append(j.text)

0          [(점프 투 파이썬 책의 커리큘럼처럼 이어져서 좋다., 0, 27, None, [])]
1         [(다만 사실상 거의 입문 표준인 파이참이나 vscode 로 에디터를 사용했으면 좋...
2                  [(입문자들은 그냥 다 따라서 하니까., 0, 19, None, [])]
3                [(파이썬이 사용되는 예제들이 많아 좋아요, 0, 21, None, [])]
4                      [(파이썬 라이브러리가 방대하다, 0, 15, None, [])]
                                ...                        
102362                  [(사진가에게 적합한 책입니다, 0, 14, None, [])]
102363                      [(기존에 봤던 책.., 0, 10, None, [])]
102364                 [(내용이 좋아 선물용으로 구입, 0, 15, None, [])]
102365                 [(정말 유용한 내용이 많습니다, 0, 15, None, [])]
102366          [(포토샵 배우기에 최고의 책인거 같습니다., 0, 22, None, [])]
Name: 0, Length: 102367, dtype: object

In [247]:
df_sep_result = pd.DataFrame(new_sep_result)

print(f'전처리 문장 : {len(df_sep_result)}개')

전처리 문장 : 101787개


In [ ]:
# df_sep_result = df_sep_result[0].str.replace('HTML5','HTML')
# df_sep_result = df_sep_result.str.replace('ES6','js')
# df_sep_result = df_sep_result.str.replace('ggplot2','ggplot')
# df_sep_result = df_sep_result.str.replace('[a-zA-Z]+[0-9]+','',regex=True) 

### 문장길이 5 이상인 경우 제거

In [244]:
df_sep_result = df_sep_result[df_sep_result.str.len() > 5]

df_sep_result = df_sep_result.reset_index(drop=True)

df_sep_result[:2]

0                          점프 투 파이썬 책의 커리큘럼처럼 이어져서 좋다.
1    다만 사실상 거의 입문 표준인 파이참이나 vscode 로 에디터를 사용했으면 좋았겠...
Name: 0, dtype: object

### 파일 저장

In [251]:
import utils as keu

## Eng to han
englist = pd.read_csv("preprocess/englist.csv")

lst = []
for i in df_sep_result[0].tolist() :
    val = keu.trans_eng_to_han(words=i,englist=englist)

    lst.append(' '.join(val))


df_sep_result = pd.DataFrame(lst)

# len 5개 이상만 포함
df_sep_result = df_sep_result[df_sep_result[0].str.len() > 5]


df_sep_result.to_csv('pre_book_reply.csv')

### 토큰 개수 확인하기

In [17]:
# token 개수 세기
tokens = tokenizer(df_sep_result['0'].tolist())

# token, len(token)
token = [[token,len(token)] for token in tokens['input_ids']]



### 최대 토큰개수 설정

In [18]:
df_token = pd.DataFrame(token)

# sen 추가
df_token['sen'] = df_sep_result['0'].tolist()

df_token.columns = ['token','num','sen']


# 3 이상 256 이하 token 개수만 남기기
df_token = df_token[(df_token.num < 256) & (df_token.num > 3) ].sort_values(by='num',ascending=False)

df_token = df_token.reset_index(drop=True)

df_token.head(3)

,token,num,sen
0,"[2, 28, 14370, 4279, 4031, 9577, 8800, 9108, 2...",250,8 할당하기 Chapter 9 태그 라벨 계정 Chapter 10 목표 달성을 위한...
1,"[2, 2634, 4112, 31727, 2399, 4568, 6517, 12504...",248,맑은 고딕 돋움 일반 응용프로그램을 포함한 많은 개발자 Apple맑은 고딕 돋움 대...
2,"[2, 9246, 17222, 6827, 13606, 4031, 9246, 4418...",243,실무 함수 제대로 익히기 실무함수 01 꼭 알아야 할 필수 함수 01 목표갑과 매출...


###   파일저장 

In [ ]:
df_sep_result.to_csv('pre_book_reply.csv')

### Generator와 Discriminator 학습을 위한 Optimizer 설정 방법

* Electra는 combined loss를 활용해 Generator와 Discriminator를 학습함.

* combined loss는 Generator loss($\mathcal{L}_{\text{MLM}}(\textbf{x}, \theta_G)$)와 Discriminator loss($ \mathcal{L}_{Disc} (\textbf{x}, \theta_{D})$)를 각각 계산한 뒤 아래의 식으로 구함.


    combined_loss = $ \min_{\theta_G, \theta_D} \sum_{\textbf{x} \in \mathcal{X}} \mathcal{L}_{\text{MLM}}(\textbf{x}, \theta_G) + \lambda \mathcal{L}_{Disc} (\textbf{x}, \theta_{D})$

### transformers에서 `save_pretrained` 매서드 사용시 학습된 모델의 weight 저장이 안되는 문제 식별

* 새로 학습한 모델을 저장한 모델의 예측 결과와 학습 이전의 원본 모델의 예측 결과가 동일함.

* 학습이 제대로 수행되지 않는 문제인지, 저장 방법의 문제인지 파악 필요


#### 학습한 모델(trained)와 기존 모델(old) 간 output 비교

In [1]:
import pandas as pd
import ast 
import torch

raw_data = pd.read_csv('data/pre_book_total_128.csv')

token = ast.literal_eval(raw_data.loc[0,'token'])

# 예제 input
input_data = torch.tensor(token).reshape(1,-1)

In [ ]:
from transformers import ElectraModel

# 모델 불러오기
old = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
trained = ElectraModel.from_pretrained('model/discriminator-iter2/')


In [6]:
old_one

tensor([[[-0.1475, -0.4093, -0.3794,  ..., -0.1255, -0.0811, -0.2957],
         [ 0.2833, -0.3376, -0.4579,  ...,  0.1145,  0.5214, -0.2951],
         [ 1.2557, -0.1452, -0.4150,  ..., -0.1571,  0.1779, -0.0560],
         ...,
         [ 0.0982, -0.2955, -0.6046,  ..., -0.3594,  0.1047, -0.0694],
         [-0.4510,  0.1381, -0.6813,  ...,  0.3969,  0.1989, -0.0629],
         [-0.1475, -0.4093, -0.3794,  ..., -0.1255, -0.0811, -0.2957]]],
       grad_fn=<NativeLayerNormBackward0>)

In [5]:
old_one = old(input_data).last_hidden_state
trained = trained(input_data).last_hidden_state

old_one == trained

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

pip install transformers

import os
!pwd
os.chdir('/content/drive/MyDrive/Colab Notebooks/electra_for_fine_tuning')
!pwd

import pandas as pd
import ast 

raw_data = pd.read_csv('data/pre_book_total_128.csv')

token = ast.literal_eval(raw_data.loc[0,'token'])

# 예제 input
input_data = torch.tensor(token).reshape(1,-1).to(device)

from transformers import ElectraModel

# 모델 불러오기
old = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator').to(device)
trained = discriminator.electra


old_one = old(input_data).last_hidden_state
trained = trained(input_data).last_hidden_state

old_one == trained

### 문제 접근 과정

* 학습 과정에 대한 문제라면 몇 번을 학습하더라도 기존 모델의 output과 동일할 것임.

* 이러한 문제가 아니라면 batch를 1회라도 학습한 모델의 output은 기존 모델의 output과 다를 것임


* 실험 결과 : batch_size 16으로 100번 학습한 모델의 output은 기존 모델의 output과 다름.
``` python
tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])
```
* 결론 : 모델을 저장하는 과정에서 학습된 모델을 저장하지 않아서 발생한 문제임.





### 모델 저장 과정에서 발생하는 문제 식별

* 기존 모델(old)의 weight와 새로 학습한 모델(new)의 weight가 동일한 문제 식별

In [ ]:
from transformers import ElectraModel

# 모델 불러오기
old = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator') #len() = 197개
new = ElectraModel.from_pretrained('model/discriminator-iter2/') #len() = 197개


In [11]:
old_param = [i for i in old.parameters()]
new_param = [i for i in new.parameters()]

# Params 비교
idx_list = []
for i in range(len(old_param)) :
    x = old_param[i] - new_param[i] # 차이가 없다면 x = 0
    if torch.sum(x.reshape(-1)).item() != 0 :
        idx_list.append(i)

idx_list 



[]

### 모델을 저장하는 매서드(`save_pretrained`)가 문제라 가정하고 접근

* huggingface는 torch 기반이므로 torch.save로 모델 저장 가능

* torch로 저장한 모듈을 불러와 params 비교 수행

* 결론 : `save_pretrained` 매서드가 문제임을 발견

In [ ]:
model_saved_by_torch = torch.load('123.pt')
old = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator') #len() = 197개

In [8]:
old_param = [i for i in old.parameters()]
new_param = [i for i in model_saved_by_torch.parameters()]

idx_list = []
for i in range(len(old_param)) :
    x = old_param[i] - new_param[i]
    if torch.sum(x.reshape(-1)).item() != 0 :
        # print(torch.sum(x.reshape(-1)).item())
        idx_list.append(i)

## 모든 param가 학습 됨을 확인함.
print(idx_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196]


### torch.load로 불러온 모델에 다시 save_pretrained로 저장
* torch 파일로 저장한 모델의 경우 weight를 transformers 모델에 저장할 수 있음.
* pre-training 과정에서는 torch로 저장한 다음 최종적으로 save_pretrained으로 저장

In [ ]:
model_saved_by_torch = ElectraModel.from_pretrained('123') # torch로 불러온 뒤 save_pretrained로 저장한 파일
old = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator') #len() = 197개

In [13]:
old_param = [i for i in old.parameters()]
new_param = [i for i in model_saved_by_torch.parameters()]

idx_list = []
for i in range(len(old_param)) :
    x = old_param[i] - new_param[i]
    if torch.sum(x.reshape(-1)).item() != 0 :
        # print(torch.sum(x.reshape(-1)).item())
        idx_list.append(i)

## 모든 param가 학습 됨을 확인함.
print(idx_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196]
